# Feature Crafting

new workbook to build the dataset from the raw data so that we can set a target for how we want to network to perform

In [1]:
import pandas as pd
import os

In [2]:
data_path = 'data'
train = 'train.csv'

In [3]:
train_data = pd.read_csv(os.path.join(data_path, train) )

/home/brian/anaconda3/envs/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# EDA Code

In [4]:
# spare space for if needed

# Feature Code
https://www.kaggle.com/cast42/xgboost-extra-features
We may need to adjust this for LSTMs as LSTMs work on sequences and cause we are dropping values we may not be getting nice sequences

In [5]:
from __future__ import print_function
'''
Public Score :  0.11727 (previous 0.11771)
Private Validation Score : [1199]	train-rmspe:0.104377	eval-rmspe:0.093786
'''

'''
Brian Score Public: 0.12378 Private: 0.14471
'''

import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb

# Gather some features
def build_features(features, data):
    # remove NaNs
    data.fillna(0, inplace=True)
    data.loc[data.Open.isnull(), 'Open'] = 1
    # Use some properties directly
    features.extend(['Store', 'CompetitionDistance', 'Promo', 'Promo2', 'SchoolHoliday'])

    # Label encode some features
    features.extend(['StoreType', 'Assortment', 'StateHoliday'])
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)

    features.extend(['DayOfWeek', 'Month', 'Day', 'Year', 'WeekOfYear'])
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    data['WeekOfYear'] = data.Date.dt.weekofyear

    # CompetionOpen en PromoOpen from https://www.kaggle.com/ananya77041/rossmann-store-sales/randomforestpython/code
    # Calculate time competition open time in months
    features.append('CompetitionOpen')
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + \
        (data.Month - data.CompetitionOpenSinceMonth)
    # Promo open time in months
    features.append('PromoOpen')
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) + \
        (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
    data.loc[data.Promo2SinceYear == 0, 'PromoOpen'] = 0

    # Indicate that sales on that day are in promo interval
    features.append('IsPromoMonth')
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['monthStr'] = data.Month.map(month2str)
    data.loc[data.PromoInterval == 0, 'PromoInterval'] = ''
    data['IsPromoMonth'] = 0
    for interval in data.PromoInterval.unique():
        if interval != '':
            for month in interval.split(','):
                data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1

    return data

## Start of main script

print("Load the training, test and store data using pandas")
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(int),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv("data/train.csv", parse_dates=[2], dtype=types)
test = pd.read_csv("data/test.csv", parse_dates=[3], dtype=types)
store = pd.read_csv("data/store.csv")

print("Assume store open, if not provided")
test.fillna(1, inplace=True)

print("Consider only open stores for training. Closed stores wont count into the score.")
train = train[train["Open"] != 0]
print("Use only Sales bigger then zero")
train = train[train["Sales"] > 0]

print("Join with store")
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

features = []

print("augment features")
train = build_features(features, train)
test = build_features([], test)
print(features)

print('training data processed')

def rmspe(y, yhat):
    return np.sqrt(np.mean(((y - yhat)/y) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

/home/brian/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Load the training, test and store data using pandas
Assume store open, if not provided
Consider only open stores for training. Closed stores wont count into the score.
Use only Sales bigger then zero
Join with store
augment features
['Store', 'CompetitionDistance', 'Promo', 'Promo2', 'SchoolHoliday', 'StoreType', 'Assortment', 'StateHoliday', 'DayOfWeek', 'Month', 'Day', 'Year', 'WeekOfYear', 'CompetitionOpen', 'PromoOpen', 'IsPromoMonth']
training data processed


# XGB Baseline

params for xgb

In [6]:
print("Train xgboost model")

params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.1,
          "max_depth": 10,
          "subsample": 0.85,
          "colsample_bytree": 0.4,
          "min_child_weight": 6,
          "silent": 1,
          "thread": 1,
          "seed": 1301
          }
num_boost_round = 1200

Train xgboost model


run model

In [7]:
print("Train a XGBoost model")
X_train, X_valid = train_test_split(train, test_size=0.012, random_state=10)
y_train = np.log1p(X_train.Sales)
y_valid = np.log1p(X_valid.Sales)
dtrain = xgb.DMatrix(X_train[features], y_train)
dvalid = xgb.DMatrix(X_valid[features], y_valid)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=200, \
  feval=rmspe_xg, verbose_eval=True)

print("Validating")
yhat = gbm.predict(xgb.DMatrix(X_valid[features]))
error = rmspe(X_valid.Sales.values, np.expm1(yhat))
print('RMSPE: {:.6f}'.format(error))

print("Make predictions on the test set")
dtest = xgb.DMatrix(test[features])
test_probs = gbm.predict(dtest)
# Make Submission
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs)})
#result.to_csv("xgboost_39_submission.csv", index=False)

Train a XGBoost model
[0]	train-rmspe:0.999526	eval-rmspe:0.999525
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 200 rounds.
[1]	train-rmspe:0.998818	eval-rmspe:0.998816
[2]	train-rmspe:0.997541	eval-rmspe:0.997538
[3]	train-rmspe:0.995383	eval-rmspe:0.995379
[4]	train-rmspe:0.99197	eval-rmspe:0.991963
[5]	train-rmspe:0.986847	eval-rmspe:0.986837
[6]	train-rmspe:0.979572	eval-rmspe:0.97956
[7]	train-rmspe:0.969721	eval-rmspe:0.969698
[8]	train-rmspe:0.956898	eval-rmspe:0.956868
[9]	train-rmspe:0.940702	eval-rmspe:0.940651
[10]	train-rmspe:0.921127	eval-rmspe:0.921067
[11]	train-rmspe:0.898091	eval-rmspe:0.897986
[12]	train-rmspe:0.871744	eval-rmspe:0.871578
[13]	train-rmspe:0.842418	eval-rmspe:0.84217
[14]	train-rmspe:0.810477	eval-rmspe:0.810119
[15]	train-rmspe:0.776565	eval-rmspe:0.776016
[16]	train-rmspe:0.740905	eval-rmspe:0.740107
[17]	train-rmspe:0.704394	eval-rmspe:0.703277
[18]	train-rmspe:

In [8]:
test_probs

array([ 8.32421017,  8.38323784,  8.46005821, ...,  9.25199795,
        8.7618084 ,  8.92962265], dtype=float32)

In [9]:
result.to_csv("submission/xgboost_baseline_submission.csv", index=False)

# Save out data to test NNs

In [10]:
import pickle
pickle.dump(X_train, open( 'feat_table/xgb_1_x_train_table.pkl', "wb" ))
pickle.dump(y_train, open( 'feat_table/xgb_1_y_train_table.pkl', "wb" ))
pickle.dump(X_valid, open( 'feat_table/xgb_1_x_valid_table.pkl', "wb" ))
pickle.dump(y_valid, open( 'feat_table/xgb_1_y_valid_table.pkl', "wb" ))
pickle.dump(test, open( 'feat_table/xgb_1_test_table.pkl', "wb" ))
pickle.dump(features, open( 'feat_table/xgb_1_features_vector_vector.pkl', "wb" ))
gbm.save_model('model/xgb_baseline.model')
